In [1]:
import os
import cv2
import numpy as np
import tkinter as tk
from tkinter import filedialog
from tkinter import messagebox
from PIL import Image, ImageTk
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import threading  # Used for running classification in the background

# --------------------------- #
# 🔹 Define Paths & Parameters
# --------------------------- #
train_dir = "output/train"
val_dir = "output/val"
input_shape = (224, 224, 3)
batch_size = 32
NUM_CLASSES = 5  # Updated number of traffic sign classes

# --------------------------- #
# 🔹 Data Preparation (for VGG16)
# --------------------------- #
train_datagen = ImageDataGenerator(rescale=1.0/255.0)
val_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_data = train_datagen.flow_from_directory(train_dir, target_size=(224, 224), 
                                               batch_size=batch_size, class_mode='categorical')
val_data = val_datagen.flow_from_directory(val_dir, target_size=(224, 224), 
                                           batch_size=batch_size, class_mode='categorical')

# --------------------------- #
# 🔹 Function to Create VGG16 Model
# --------------------------- #
def create_vgg16_model():
    base_model = VGG16(weights="imagenet", include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Freeze pre-trained layers

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.5)(x)
    output = Dense(NUM_CLASSES, activation="softmax")(x)  # Ensure correct number of classes

    model = Model(inputs=base_model.input, outputs=output)
    return model

# --------------------------- #
# 🔹 Train and Save VGG16 Model (if not exists)
# --------------------------- #
vgg16_model_path = "vgg16_model.h5"

if not os.path.exists(vgg16_model_path):
    print("Training VGG16 Model... 🚀")
    vgg16_model = create_vgg16_model()
    vgg16_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                        loss="categorical_crossentropy",
                        metrics=["accuracy"])
    
    vgg16_model.fit(train_data, epochs=30, validation_data=val_data)
    vgg16_model.save(vgg16_model_path)
    print(f"✅ VGG16 Model saved as '{vgg16_model_path}'.")
else:
    print(f"✅ Found existing model '{vgg16_model_path}', skipping training.")

# Load the trained model
vgg16_model = create_vgg16_model()
vgg16_model.load_weights(vgg16_model_path)
print("✅ VGG16 Model Loaded Successfully!")

# --------------------------- #
# 🔹 Function: Preprocess Image for VGG16
# --------------------------- #
def preprocess_for_vgg16(image):
    image = image.resize((224, 224))  # Resize image
    image = np.array(image) / 255.0  # Normalize
    image = np.expand_dims(image, axis=0)  # Expand dims for model
    return image

# --------------------------- #
# 🔹 Function: Classify Image Using VGG16
# --------------------------- #
def classify_image(image_path):
    image = Image.open(image_path)  # Open image
    processed_img = preprocess_for_vgg16(image)  # Preprocess for VGG16
    predictions = vgg16_model.predict(processed_img)  # Predict

    class_id = np.argmax(predictions)
    confidence = predictions[0][class_id]
    
    # Class labels
    class_labels = ['100_new', '200_new', '20_new', '500_new', '50_new']
    label = f"{class_labels[class_id]} ({confidence:.2f})"
    
    return label

# --------------------------- #
# 🔹 Function to Open File Dialog and Select Image
# --------------------------- #
def select_image():
    root = tk.Tk()
    root.withdraw()  # Hide the root window
    image_path = filedialog.askopenfilename(title="Select Image", filetypes=[("Image files", "*.jpg;*.jpeg;*.png")])
    return image_path

# --------------------------- #
# 🔹 Function to Display the Classification Result
# --------------------------- #
def show_classification_result(image_path):
    result_label.config(text="Processing... Please wait.", fg="black")
    
    # Run classification in a background thread to avoid blocking UI
    threading.Thread(target=run_classification, args=(image_path,)).start()

# --------------------------- #
# 🔹 Function to Run Classification in the Background
# --------------------------- #
def run_classification(image_path):
    try:
        # Classify the uploaded image
        result = classify_image(image_path)
        
        # Update the result label in the main thread
        result_label.config(text=f"Prediction: {result}", fg="green")
        
    except Exception as e:
        # In case of error, display the error message
        result_label.config(text=f"Error: {str(e)}", fg="red")

# --------------------------- #
# 🔹 Function to Load and Display Image in Tkinter
# --------------------------- #
def load_image(image_path):
    image = Image.open(image_path)
    image.thumbnail((400, 400))  # Resize for better display
    img_tk = ImageTk.PhotoImage(image)
    
    # Keep a reference of the image to prevent it from being garbage collected
    preview_label.image = img_tk  
    preview_label.config(image=img_tk)

# --------------------------- #
# 🔹 Create the Tkinter GUI
# --------------------------- #
window = tk.Tk()
window.title("Image Classification")
window.geometry("600x600")

# Create an upload button
upload_button = tk.Button(window, text="Upload Image", font=("Helvetica", 14), command=lambda: on_upload())
upload_button.pack(pady=10)

# Label to display the selected image preview
preview_label = tk.Label(window)
preview_label.pack(pady=20)

# Label for the classification result
result_label = tk.Label(window, text="Prediction: None", font=("Helvetica", 14), fg="blue")
result_label.pack(pady=20)

# --------------------------- #
# 🔹 Upload Button Logic
# --------------------------- #
def on_upload():
    image_path = select_image()
    if image_path:
        load_image(image_path)
        show_classification_result(image_path)
    else:
        messagebox.showerror("Error", "No image selected. Please upload an image.")

# Run the Tkinter event loop
window.mainloop()



Found 375 images belonging to 5 classes.
Found 100 images belonging to 5 classes.
✅ Found existing model 'vgg16_model.h5', skipping training.


✅ VGG16 Model Loaded Successfully!


In [1]:
import cv2
import numpy as np
import tensorflow as tf
from ultralytics import YOLO
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D

# Load the YOLOv8 model
yolo_model = YOLO("yolov8s.pt")
# Configure model to only detect currency notes
yolo_model.classes = [67]  # Class ID for currency notes

# --------------------------- #
# 🔹 Load VGG16 Model for Currency Classification
# --------------------------- #
def create_vgg16_model():
    input_shape = (224, 224, 3)
    base_model = VGG16(weights="imagenet", include_top=False, input_shape=input_shape)
    base_model.trainable = False

    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.5)(x)
    output = Dense(5, activation="softmax")(x)

    model = Model(inputs=base_model.input, outputs=output)
    return model

vgg16_model = create_vgg16_model()
vgg16_model.load_weights("vgg16_model.h5")

# Class labels for currency notes
class_labels = ['100_new', '200_new', '20_new', '500_new', '50_new']

def preprocess_for_vgg16(cropped_img):
    if cropped_img.size == 0:
        return None
    try:
        cropped_img = cv2.resize(cropped_img, (224, 224))
        cropped_img = cropped_img.astype("float32") / 255.0
        cropped_img = np.expand_dims(cropped_img, axis=0)
        return cropped_img
    except Exception as e:
        print(f"Error in preprocessing: {e}")
        return None

def is_valid_currency_note(box, conf, min_area=5000):
    # Get box dimensions
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    area = (x2 - x1) * (y2 - y1)
    aspect_ratio = (x2 - x1) / (y2 - y1)
    
    # Check if the note meets our criteria
    return (conf > 0.6 and  # High confidence
            area >= min_area and  # Minimum size
            1.5 <= aspect_ratio <= 3.0)  # Typical currency note aspect ratio

def real_time_detection_and_classification():
    cap = cv2.VideoCapture(0)
    
    # Set camera properties for better performance
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    cap.set(cv2.CAP_PROP_FPS, 30)

    print("Starting detection. Press 'q' to quit.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Create a copy for display
        display_frame = frame.copy()

        # Run YOLO detection
        results = yolo_model(frame, conf=0.5)

        best_note = None
        highest_conf = 0

        for result in results:
            boxes = result.boxes
            for box in boxes:
                conf = float(box.conf[0])
                
                # Check if this is a valid currency note with higher confidence
                if is_valid_currency_note(box, conf):
                    if conf > highest_conf:
                        highest_conf = conf
                        best_note = box

        # Process only the best detected note
        if best_note is not None:
            x1, y1, x2, y2 = map(int, best_note.xyxy[0])
            cropped_obj = frame[y1:y2, x1:x2]
            processed_img = preprocess_for_vgg16(cropped_obj)

            if processed_img is not None:
                try:
                    predictions = vgg16_model.predict(processed_img, verbose=0)
                    note_class = np.argmax(predictions)
                    note_conf = predictions[0][note_class]

                    if note_conf > 0.7:  # High confidence threshold for currency
                        label = f"{class_labels[note_class]} ({note_conf:.2f})"
                        cv2.rectangle(display_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        cv2.putText(display_frame, label, (x1, y1 - 10),
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                except Exception as e:
                    print(f"Error in classification: {e}")

        # Add FPS counter
        fps = cap.get(cv2.CAP_PROP_FPS)
        cv2.putText(display_frame, f"FPS: {int(fps)}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display the frame
        cv2.imshow("Currency Detection", display_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    real_time_detection_and_classification()




Starting detection. Press 'q' to quit.

0: 480x640 1 person, 564.2ms
Speed: 17.4ms preprocess, 564.2ms inference, 17.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 280.2ms
Speed: 2.0ms preprocess, 280.2ms inference, 1.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 279.1ms
Speed: 36.1ms preprocess, 279.1ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 265.6ms
Speed: 5.0ms preprocess, 265.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 255.7ms
Speed: 7.0ms preprocess, 255.7ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 238.5ms
Speed: 3.0ms preprocess, 238.5ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 295.7ms
Speed: 4.0ms preprocess, 295.7ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 279.7ms
Speed: 4.5ms preprocess, 279.